In [32]:
import statistics
import timeit
import itertools
import numpy as np
import torch
import torch.nn as nn
import optuna
import sklearn.metrics as skm
import importlib
import fastai
import tsai
import torch.nn.functional as F
import copy
import shap
importlib.reload(fastai)
importlib.reload(tsai)
import signatory
import timeit

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
import Data_load_neat as Data_load
import LM_cv_neat


from collections import Counter
from tsai.all import *
from tsai.data.validation import combine_split_data, get_splits
from tsai.models.InceptionTimePlus import InceptionTimePlus
from tsai.tslearner import TSClassifier
from optuna.samplers import TPESampler
from optuna.integration import FastAIPruningCallback
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from torch.utils.data import WeightedRandomSampler
from fastai.vision.all import *
from fastai.callback.tracker import EarlyStoppingCallback, ReduceLROnPlateau
from fastai.data.transforms import Categorize
from fastai.losses import BCEWithLogitsLossFlat, FocalLoss, FocalLossFlat
from fastai.metrics import accuracy, BrierScore, F1Score, RocAucBinary


import Data_load_neat as Data_load
import MLmodel_opt_learner_neat as MLmodel_opt_learner

In [33]:
# load in arguments from command line
name = "data_2real5newbigdet"
model_name="ResNet"
randnum_split=3
epochs=8
device = 1
filepath="/home/DIDE/smishra/Simulations/"
folds=5
randnum=1

In [34]:
X_raw, y_raw = Data_load.load_data(name=name,filepath=filepath)

## Function to obtain the train/test split
X_trainvalid, Y_trainvalid, X_test, Y_test, splits = Data_load.split_data(X=X_raw,Y=y_raw,randnum=randnum_split)

## Now scale all the data for ease (can fix this later)
X_scaled=Data_load.prep_data(X_raw,splits)

X_trainvalid_s, X_test_s=X_scaled[splits[0]], X_scaled[splits[1]]


(100000, 9, 40) (100000, 40) (100000, 1)
(100000,)
Random state set:3, cuda used: True
Counter({0.0: 93055, 1.0: 6945}) Counter({0.0: 74444, 1.0: 5556}) Counter({0.0: 18611, 1.0: 1389})


In [35]:
def metrics_bin(pred, y_test):
    # function to output accuracy, precision, recall, f1-score, AUC score and area under precision-recall curve and classification matrix
    acc = accuracy_score(y_test,pred)
    prec = precision_score(y_test,pred)
    rec = recall_score( y_test,pred)
    fone = f1_score(y_test,pred)
    auc = roc_auc_score(y_test, pred)
    prc= average_precision_score(y_test,pred)
    print("{:<40} {:.6f}".format("Accuracy:", acc))
    print("{:<40} {:.6f}".format("Precision:", prec))
    print("{:<40} {:.6f}".format("Recall:", rec))
    print("{:<40} {:.6f}".format("F1 score:", fone))
    print("{:<40} {:.6f}".format("AUC score:", auc))

    LR00 = np.sum(pred[(pred == y_test) & (y_test == 0)] + 1)
    LR10 = np.sum(pred[(pred != y_test) & (y_test == 1)] + 1)
    LR01 = np.sum(pred[(pred != y_test) & (y_test == 0)])
    LR11 = np.sum(pred[(pred == y_test) & (y_test == 1)])
    print("{:<40} {:.6f}".format("Predicted 0 when actually 0:", LR00))
    print("{:<40} {:.6f}".format("Predicted 0 when actually 1:", LR10))
    print("{:<40} {:.6f}".format("Predicted 1 when actually 0:", LR01))
    print("{:<40} {:.6f}".format("Predicted 1 when actually 1:", LR11))
    return acc, prec, rec, fone, auc, prc, LR00, LR01, LR10, LR11


In [36]:
import LM_cv_neat
from sklearn.linear_model import LogisticRegression

In [37]:
## lr just for final row
X_LRtrain=np.squeeze(X_trainvalid[:,-1,:])
X_LRtest=np.squeeze(X_test[:,-1,:])


LRmodel = LogisticRegression(penalty="l1", tol=0.01, solver="saga",random_state=randnum).fit(X_LRtrain, Y_trainvalid)

LRpred = LRmodel.predict(X_LRtest)


acc, prec, rec, fone, auc, prc, LR00, LR01, LR10, LR11 =LM_cv_neat.metrics_bin(LRpred, Y_test)

LRmodel.coef_

Accuracy:                                1.000000
Precision:                               1.000000
Recall:                                  1.000000
F1 score:                                1.000000
AUC score:                               1.000000
Predicted 0 when actually 0:             18611.000000
Predicted 0 when actually 1:             0.000000
Predicted 1 when actually 0:             0.000000
Predicted 1 when actually 1:             1389.000000


array([[-1.10222931e+01, -1.12651005e+01, -1.09118872e+01,
        -1.08828640e+01, -1.08392677e+01, -1.08580055e+01,
        -1.07901697e+01, -1.08152561e+01, -1.08252182e+01,
        -1.08437624e+01, -1.08936977e+01, -1.08736610e+01,
        -1.08684416e+01, -1.08235626e+01, -1.08113194e+01,
        -1.08230591e+01, -1.08848696e+01, -1.08040190e+01,
        -1.08010397e+01, -1.08326216e+01, -4.25344624e-05,
         0.00000000e+00,  0.00000000e+00,  5.78321169e-06,
        -4.51730120e-05,  5.31642718e-06, -4.24342979e-05,
         5.90219952e-06,  4.92972367e-06,  4.87788157e-07,
         0.00000000e+00, -2.19462936e-06,  0.00000000e+00,
         0.00000000e+00,  3.02286435e-06, -4.60505653e-05,
         0.00000000e+00, -3.30966941e-07,  0.00000000e+00,
         0.00000000e+00]], dtype=float32)

In [38]:
def Sig_func(X_trainvalid, X_test, K):

    num_samples,num_features,num_timepoints = np.shape(X_trainvalid)
    num_samples_test = np.shape(X_test)[0]
    sig_length=int(((num_features+1)**(K+1)-1)/num_features ) -1
    X_sigtrainvalid = np.zeros(shape=[num_samples, 1, sig_length], dtype=float)
    X_sigtest = np.zeros(shape=[num_samples_test, 1, sig_length], dtype=float)
    

    for i in range(0, num_samples):
        xt = np.arange(0, num_timepoints)  ## time/age
        xtT = torch.tensor(xt.reshape(num_timepoints, 1), dtype=torch.float)
        path = copy.copy(xtT)
        for j in range(0, num_features):
        
            x1 = X_trainvalid[i, j, :]
            x1T = torch.tensor(x1.reshape(num_timepoints, 1), dtype=torch.float)
            path = torch.cat((path,x1T), 1)

            #path = torch.cat((xtT, x1T, x2T, x3T, x4T, x5T, x6T, x7T, x8T, x9T), 1)
        path2 = path.unsqueeze(0)
        X_sigtrainvalid[i, 0, :] = signatory.signature(path2, K).numpy()

    for i in range(0, num_samples_test):
        xt = np.arange(0, num_timepoints)  ## time/age
        xtT = torch.tensor(xt.reshape(num_timepoints, 1), dtype=torch.float)
        path = copy.copy(xtT)
        for j in range(0, num_features):
        
            x1 = X_test[i, j, :]
            x1T = torch.tensor(x1.reshape(num_timepoints, 1), dtype=torch.float)
            path = torch.cat((path,x1T), 1)

            #path = torch.cat((xtT, x1T, x2T, x3T, x4T, x5T, x6T, x7T, x8T, x9T), 1)
        path2 = path.unsqueeze(0)
        X_sigtest[i, 0, :] = signatory.signature(path2, K).numpy()

    # function to reshape the data to flatten the time/feature dimensions into one
    X_LRtrainvalid=np.reshape(X_sigtrainvalid,(num_samples,sig_length))
    X_LRtest=np.reshape(X_sigtest,(num_samples_test,sig_length))

    return X_LRtrainvalid, X_LRtest



def SIGmodel_block(X_trainvalid, Y_trainvalid, X_test, Y_test, sig_depth, K, randnum=8):
    # function to fit and analyse the logistic regression model
    
    # random seed
    Data_load.random_seed(randnum)

    # scale and one-hot the data
    #X_scaled=Data_load.prep_data(X, splits)
    #XStrainvalid=X_scaled[splits[0]]
    #XStest=X_scaled[splits[1]]

    # flatten the data
    X_LRtrain, X_LRtest = Sig_func(X_trainvalid, X_test, sig_depth, K)

    # fit the logistic regression model to the train data
    start = timeit.default_timer()
    LRmodel = LogisticRegression(penalty="l1", tol=0.01, solver="saga",random_state=randnum).fit(X_LRtrain, Y_trainvalid)
    stop = timeit.default_timer()
    runtime=stop - start

    # get model predictions on the test data
    LRpred = LRmodel.predict(X_LRtest)

    # get output metrics for test data
    acc, prec, rec, fone, auc, prc, LR00, LR01, LR10, LR11= metrics_bin(LRpred, Y_test)
    return runtime, acc, prec, rec, fone, auc, prc,  LR00, LR01, LR10, LR11

In [39]:
import copy

In [10]:

K=3

num_samples,num_features,num_timepoints = np.shape(X_trainvalid)
num_samples_test = np.shape(X_test)[0]
sig_length=int(((num_features+1)**(K+1)-1)/num_features ) -1

X_sigtrainvalid = np.zeros(shape=[num_samples, 1, sig_length], dtype=float)
X_sigtest = np.zeros(shape=[num_samples_test, 1, sig_length], dtype=float)


for i in range(0, num_samples):
    xt = np.arange(0, num_timepoints)  ## time/age
    xtT = torch.tensor(xt.reshape(num_timepoints, 1), dtype=torch.float)
    path = copy.copy(xtT)
    for j in range(0, num_features):
    
        x1 = X_trainvalid[i, j, :]
        x1T = torch.tensor(x1.reshape(num_timepoints, 1), dtype=torch.float)
        path = torch.cat((path,x1T), 1)

        #path = torch.cat((xtT, x1T, x2T, x3T, x4T, x5T, x6T, x7T, x8T, x9T), 1)
    path2 = path.unsqueeze(0)
    X_sigtrainvalid[i, 0, :] = signatory.signature(path2, K).numpy()

for i in range(0, num_samples_test):
    xt = np.arange(0, num_timepoints)  ## time/age
    xtT = torch.tensor(xt.reshape(num_timepoints, 1), dtype=torch.float)
    path = copy.copy(xtT)
    for j in range(0, num_features):
    
        x1 = X_test[i, j, :]
        x1T = torch.tensor(x1.reshape(num_timepoints, 1), dtype=torch.float)
        path = torch.cat((path,x1T), 1)

        #path = torch.cat((xtT, x1T, x2T, x3T, x4T, x5T, x6T, x7T, x8T, x9T), 1)
    path2 = path.unsqueeze(0)
    X_sigtest[i, 0, :] = signatory.signature(path2, K).numpy()

# function to reshape the data to flatten the time/feature dimensions into one
X_LRtrainvalid=np.reshape(X_sigtrainvalid,(num_samples,sig_length))
X_LRtest=np.reshape(X_sigtest,(num_samples_test,sig_length))

In [40]:
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, accuracy_score, recall_score, precision_score

In [41]:
K=3

X_LRtrain, X_LRtest = Sig_func(X_trainvalid, X_test, K=K)

# fit the logistic regression model to the train data
LRmodel = LogisticRegression(penalty="l1", tol=0.01, solver="saga",random_state=randnum).fit(X_LRtrain, Y_trainvalid)

# get model predictions on the test data
LRpred = LRmodel.predict(X_LRtest)

# get output metrics for test data
acc, prec, rec, fone, auc, prc, LR00, LR01, LR10, LR11= metrics_bin(LRpred, Y_test)

Accuracy:                                0.929500
Precision:                               0.376471
Recall:                                  0.023038
F1 score:                                0.043419
AUC score:                               0.510095
Predicted 0 when actually 0:             18558.000000
Predicted 0 when actually 1:             1357.000000
Predicted 1 when actually 0:             53.000000
Predicted 1 when actually 1:             32.000000


In [42]:
X_LRtrain.shape


(80000, 1110)

In [43]:
X_trainvalid.shape

(80000, 9, 40)

In [44]:
X_trainvalid[0,:,:]

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
   

In [46]:
from scipy.interpolate import RegularGridInterpolator

In [48]:
def f(x, y, z):
    return 2 * x**3 + 3 * y**2 - z
x = np.linspace(1, 4, 11)
y = np.linspace(4, 7, 22)
z = np.linspace(7, 9, 33)
xg, yg ,zg = np.meshgrid(x, y, z, indexing='ij', sparse=True)
data = f(xg, yg, zg)

print(data.shape)


(11, 22, 33)


In [50]:
x

array([1. , 1.3, 1.6, 1.9, 2.2, 2.5, 2.8, 3.1, 3.4, 3.7, 4. ])

In [49]:
interp = RegularGridInterpolator((x, y, z), data)

pts = np.array([[2.1, 6.2, 8.3],
                [3.3, 5.2, 7.1]])
interp(pts)

array([125.80469388, 146.30069388])

In [106]:
x = range(1,10)
y = range(1,41)
xg, yg = np.meshgrid(x, y, indexing='ij', sparse=True)
data = X_trainvalid[0,:,:]
interp = RegularGridInterpolator((x, y), data)

pts = np.array([[9, 1],
                [9, 2],
                [9, 3],
                [9, 4],
                [9, 5],
                [9, 6],
                [9, 7],
                [9, 8],
                [9, 9],
                [9, 10],
                [9, 11]])
print(interp(pts))
pts = np.array([[9, 1],
                [9, 1.5],
                [9, 2],
                [9, 2.5],
                [9, 3],
                [9, 3.5],
                [9, 4],
                [9, 4.5],
                [9, 5],
                [9, 5.5],
                [9, 6]])
print(interp(pts))

num_newfeatures = 100
xn = np.linspace(1,9,9)
yn= np.linspace(1,40,num_newfeatures)

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
[0.  0.  0.  0.5 1.  0.5 0.  0.  0.  0.  0. ]


In [107]:
xng, yng = np.meshgrid(xn, yn)


In [136]:
num_samples, num_features, num_timepoints = X_trainvalid.shape
x = np.linspace(0,num_features-1,num_features)
y= np.linspace(0,num_timepoints-1,num_timepoints)
xg, yg = np.meshgrid(x, y, indexing='ij', sparse=True)
data = X_trainvalid[0,:,:]
interp = RegularGridInterpolator((x, y), data)

In [137]:
x

array([0., 1., 2., 3., 4., 5., 6., 7., 8.])

In [138]:
X_tv_sig= np.zeros(shape=(num_samples,num_features,num_newfeatures))
#x = range(0,num_features)
#y = range(0,num_timepoints)
xn = np.linspace(0,num_features-1,num_features)
yn= np.linspace(1,num_timepoints-1,num_newfeatures)
xng, yng = np.meshgrid(xn, yn)

In [155]:
xng.shape

(100, 9)

In [156]:
for j in x:
    for k in y:
        print([k,j])
        print(xng[int(k),int(j)])

[0.0, 0.0]
0.0
[1.0, 0.0]
0.0
[2.0, 0.0]
0.0
[3.0, 0.0]
0.0
[4.0, 0.0]
0.0
[5.0, 0.0]
0.0
[6.0, 0.0]
0.0
[7.0, 0.0]
0.0
[8.0, 0.0]
0.0
[9.0, 0.0]
0.0
[10.0, 0.0]
0.0
[11.0, 0.0]
0.0
[12.0, 0.0]
0.0
[13.0, 0.0]
0.0
[14.0, 0.0]
0.0
[15.0, 0.0]
0.0
[16.0, 0.0]
0.0
[17.0, 0.0]
0.0
[18.0, 0.0]
0.0
[19.0, 0.0]
0.0
[20.0, 0.0]
0.0
[21.0, 0.0]
0.0
[22.0, 0.0]
0.0
[23.0, 0.0]
0.0
[24.0, 0.0]
0.0
[25.0, 0.0]
0.0
[26.0, 0.0]
0.0
[27.0, 0.0]
0.0
[28.0, 0.0]
0.0
[29.0, 0.0]
0.0
[30.0, 0.0]
0.0
[31.0, 0.0]
0.0
[32.0, 0.0]
0.0
[33.0, 0.0]
0.0
[34.0, 0.0]
0.0
[35.0, 0.0]
0.0
[36.0, 0.0]
0.0
[37.0, 0.0]
0.0
[38.0, 0.0]
0.0
[39.0, 0.0]
0.0
[0.0, 1.0]
1.0
[1.0, 1.0]
1.0
[2.0, 1.0]
1.0
[3.0, 1.0]
1.0
[4.0, 1.0]
1.0
[5.0, 1.0]
1.0
[6.0, 1.0]
1.0
[7.0, 1.0]
1.0
[8.0, 1.0]
1.0
[9.0, 1.0]
1.0
[10.0, 1.0]
1.0
[11.0, 1.0]
1.0
[12.0, 1.0]
1.0
[13.0, 1.0]
1.0
[14.0, 1.0]
1.0
[15.0, 1.0]
1.0
[16.0, 1.0]
1.0
[17.0, 1.0]
1.0
[18.0, 1.0]
1.0
[19.0, 1.0]
1.0
[20.0, 1.0]
1.0
[21.0, 1.0]
1.0
[22.0, 1.0]
1.0
[23.0, 1.0]


In [186]:
for i in range(0,num_samples):
    data = X_trainvalid[i,:,:]
    interp = RegularGridInterpolator((x, y), data)
    for j in xn:
        for k in yn:
            X_tv_sig[int(i),int(j),int(k)] = interp([xng[int(k),int(j)],yng[int(k),int(j)]])



In [187]:
i=0
data = X_trainvalid[i,:,:]
interp = RegularGridInterpolator((x, y), data)

In [184]:
x

array([0., 1., 2., 3., 4., 5., 6., 7., 8.])

In [188]:
yn

array([ 1.        ,  1.38383838,  1.76767677,  2.15151515,  2.53535354,
        2.91919192,  3.3030303 ,  3.68686869,  4.07070707,  4.45454545,
        4.83838384,  5.22222222,  5.60606061,  5.98989899,  6.37373737,
        6.75757576,  7.14141414,  7.52525253,  7.90909091,  8.29292929,
        8.67676768,  9.06060606,  9.44444444,  9.82828283, 10.21212121,
       10.5959596 , 10.97979798, 11.36363636, 11.74747475, 12.13131313,
       12.51515152, 12.8989899 , 13.28282828, 13.66666667, 14.05050505,
       14.43434343, 14.81818182, 15.2020202 , 15.58585859, 15.96969697,
       16.35353535, 16.73737374, 17.12121212, 17.50505051, 17.88888889,
       18.27272727, 18.65656566, 19.04040404, 19.42424242, 19.80808081,
       20.19191919, 20.57575758, 20.95959596, 21.34343434, 21.72727273,
       22.11111111, 22.49494949, 22.87878788, 23.26262626, 23.64646465,
       24.03030303, 24.41414141, 24.7979798 , 25.18181818, 25.56565657,
       25.94949495, 26.33333333, 26.71717172, 27.1010101 , 27.48

In [189]:
numt_samples, numt_features, numt_timepoints = X_test.shape
X_t_sig= np.zeros(shape=(numt_samples,num_features,num_newfeatures))

In [190]:
for i in range(0,numt_samples):
    data = X_test[i,:,:]
    interp = RegularGridInterpolator((x, y), data)
    for j in x:
        for k in y:
            X_t_sig[int(i),int(j),int(k)] = interp([xng[int(k),int(j)],yng[int(k),int(j)]])


In [191]:
K=3

X_LRtrain, X_LRtest = Sig_func(X_tv_sig, X_t_sig, K=K)

# fit the logistic regression model to the train data
LRmodel = LogisticRegression(penalty="l1", tol=0.01, solver="saga",random_state=randnum).fit(X_LRtrain, Y_trainvalid)

# get model predictions on the test data
LRpred = LRmodel.predict(X_LRtest)

# get output metrics for test data
acc, prec, rec, fone, auc, prc, LR00, LR01, LR10, LR11= metrics_bin(LRpred, Y_test)

Accuracy:                                0.930550
Precision:                               0.000000
Recall:                                  0.000000
F1 score:                                0.000000
AUC score:                               0.500000
Predicted 0 when actually 0:             18611.000000
Predicted 0 when actually 1:             1389.000000
Predicted 1 when actually 0:             0.000000
Predicted 1 when actually 1:             0.000000


In [192]:
X_tv_sig[0,8,:]

array([0.        , 0.38383838, 0.76767677, 0.84848485, 0.46464646,
       0.08080808, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.06060606, 0.44444444, 0.82828283, 0.78787879,
       0.4040404 , 0.02020202, 0.        , 0.        , 0.13131313,
       0.51515152, 0.8989899 , 0.71717172, 0.33333333, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [193]:
X_trainvalid[0,8,:]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0.], dtype=float32)

In [12]:
K=2

X_LRtrain, X_LRtest = Sig_func(X_trainvalid, X_test, K=K)


In [13]:
import sklearn

In [14]:
class_weights=sklearn.utils.class_weight.compute_class_weight(class_weight='balanced', classes=[0,1], y=Y_trainvalid)
print(class_weights)

[0.53731664 7.19942405]


In [15]:

# fit the logistic regression model to the train data
LRmodel = LogisticRegression(penalty="l2", solver="saga",random_state=randnum,multi_class='ovr',class_weight=class_weights).fit(X_LRtrain, Y_trainvalid)

# get model predictions on the test data
LRpred = LRmodel.predict(X_LRtest)

# get output metrics for test data
acc, prec, rec, fone, auc, prc, LR00, LR01, LR10, LR11= metrics_bin(LRpred, Y_test)

Accuracy:                                0.930550
Precision:                               0.000000
Recall:                                  0.000000
F1 score:                                0.000000
AUC score:                               0.500000
Predicted 0 when actually 0:             18611.000000
Predicted 0 when actually 1:             1389.000000
Predicted 1 when actually 0:             0.000000
Predicted 1 when actually 1:             0.000000


In [16]:
K=2

X_LRtrain, X_LRtest = Sig_func(X_trainvalid[:,range(7,9),:], X_test[:,range(7,9),:], K=K)

In [17]:

# fit the logistic regression model to the train data
LRmodel = LogisticRegression(penalty="l2", solver="saga",random_state=randnum,multi_class='ovr',class_weight=class_weights).fit(X_LRtrain, Y_trainvalid)

# get model predictions on the test data
LRpred = LRmodel.predict(X_LRtest)

# get output metrics for test data
acc, prec, rec, fone, auc, prc, LR00, LR01, LR10, LR11= metrics_bin(LRpred, Y_test)

Accuracy:                                0.930550
Precision:                               0.000000
Recall:                                  0.000000
F1 score:                                0.000000
AUC score:                               0.500000
Predicted 0 when actually 0:             18611.000000
Predicted 0 when actually 1:             1389.000000
Predicted 1 when actually 0:             0.000000
Predicted 1 when actually 1:             0.000000


In [25]:
X_LRtrain, X_LRtest = LM_cv_neat.LM_func(X_trainvalid, X_test)

In [27]:
from sklearn.preprocessing import PolynomialFeatures

In [29]:

poly = PolynomialFeatures(2, interaction_only=True)
Xpoly=poly.fit_transform(X_LRtrain)

Xpoly.shape

(80000, 64981)

In [30]:
LRmodel = LogisticRegression(penalty="l1", tol=0.01, solver="saga",random_state=randnum).fit(Xpoly, Y_trainvalid)

X_test_poly=poly.transform(X_LRtest)
print(X_test_poly.shape)

   # get model predictions on the test data
LRpred = LRmodel.predict(X_test_poly)

   # get output metrics for test data
acc, prec, rec, fone, auc, prc, LR00, LR01, LR10, LR11 =metrics_bin(LRpred, Y_test)

(20000, 64981)
Accuracy:                                0.992200
Precision:                               1.000000
Recall:                                  0.887689
F1 score:                                0.940503
AUC score:                               0.943844
Predicted 0 when actually 0:             18611.000000
Predicted 0 when actually 1:             156.000000
Predicted 1 when actually 0:             0.000000
Predicted 1 when actually 1:             1233.000000


In [23]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

In [31]:
selector = RFE(LogisticRegression(), 25)
final_clf = SVC()
rfe_model = Pipeline([("rfe",selector),('model',final_clf)])

TypeError: __init__() takes 2 positional arguments but 3 were given

In [ ]:
rfe_model.fit(Xpoly,Y_trainvalid)

In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(X_LRtrain, Y_trainvalid)

Y_pred_xgb = xgb_model.predict(X_test)

In [118]:
K=3

X_LRtrain, X_LRtest = Sig_func(X_trainvalid[:,range(7,9),:], X_test[:,range(7,9),:], K=K)


# fit the logistic regression model to the train data
LRmodel = LogisticRegression(penalty="l2", solver="saga",random_state=randnum,multi_class='ovr',class_weight=class_weights).fit(X_LRtrain, Y_trainvalid)

# get model predictions on the test data
LRpred = LRmodel.predict(X_LRtest)

# get output metrics for test data
acc, prec, rec, fone, auc, prc, LR00, LR01, LR10, LR11= metrics_bin(LRpred, Y_test)

Accuracy:                                0.929250
Precision:                               0.379630
Recall:                                  0.029518
F1 score:                                0.054776
AUC score:                               0.512959
Predicted 0 when actually 0:             18544.000000
Predicted 0 when actually 1:             1348.000000
Predicted 1 when actually 0:             67.000000
Predicted 1 when actually 1:             41.000000


In [123]:
K=3 
num_samples,num_features,num_timepoints = np.shape(X_trainvalid)
num_features=1
num_samples_test = np.shape(X_test)[0]
sig_length=int(((num_features+1)**(K+1)-1)/num_features ) -1
X_sigtrainvalid = np.zeros(shape=[num_samples, 1, sig_length], dtype=float)
X_sigtest = np.zeros(shape=[num_samples_test, 1, sig_length], dtype=float)


for i in range(0, num_samples):
    xt = np.arange(0, num_timepoints)  ## time/age
    xtT = torch.tensor(xt.reshape(num_timepoints, 1), dtype=torch.float)

    x1 = X_trainvalid[i, 8, :]
    x1T = torch.tensor(x1.reshape(num_timepoints, 1), dtype=torch.float)
    path = torch.cat((xtT,x1T), 1)

    path2 = path.unsqueeze(0)
    X_sigtrainvalid[i, 0, :] = signatory.signature(path2, K).numpy()

for i in range(0, num_samples_test):
    xt = np.arange(0, num_timepoints)  ## time/age
    xtT = torch.tensor(xt.reshape(num_timepoints, 1), dtype=torch.float)
    
    x1 = X_test[i, 8, :]
    x1T = torch.tensor(x1.reshape(num_timepoints, 1), dtype=torch.float)
    path = torch.cat((xtT,x1T), 1)

    path2 = path.unsqueeze(0)
    X_sigtest[i, 0, :] = signatory.signature(path2, K).numpy()

# function to reshape the data to flatten the time/feature dimensions into one
X_LRtrainvalid=np.reshape(X_sigtrainvalid,(num_samples,sig_length))
X_LRtest=np.reshape(X_sigtest,(num_samples_test,sig_length))

In [124]:
LRmodel = LogisticRegression(penalty="l2", solver="saga",random_state=randnum,multi_class='ovr',class_weight=class_weights).fit(X_LRtrainvalid, Y_trainvalid)

# get model predictions on the test data
LRpred = LRmodel.predict(X_LRtest)

# get output metrics for test data
acc, prec, rec, fone, auc, prc, LR00, LR01, LR10, LR11= metrics_bin(LRpred, Y_test)

Accuracy:                                0.929400
Precision:                               0.394495
Recall:                                  0.030958
F1 score:                                0.057410
AUC score:                               0.513706
Predicted 0 when actually 0:             18545.000000
Predicted 0 when actually 1:             1346.000000
Predicted 1 when actually 0:             66.000000
Predicted 1 when actually 1:             43.000000


In [125]:
unique_rowsmin, unique_rowsmin_cnt = np.unique(X_LRtrainvalid, axis=0,return_counts=True)
np.unique(unique_rowsmin_cnt,return_counts=True)

(array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  66,
         67,  68,  69,  70,  72,  73,  75,  76,  77,  78,  79,  80,  81,
         82,  83,  86,  87,  88,  91,  92,  93,  95,  96,  99, 100, 102,
        115, 117, 461]),
 array([26948,  4272,  1688,   848,   510,   347,   245,   189,   155,
          139,   104,    91,    70,    70,    60,    43,    41,    28,
           28,    24,    27,    23,    17,    11,    14,    11,    10,
            8,    13,    13,     6,     8,    13,    10,    10,     3,
            4,     5,     7,    10,     5,     7,     4,    13,     8,
           11,     3,     6,     5,     7,     8,    11,     5,     5,
            4,     4,     3,     7,   

In [126]:
unique_rowsmain, unique_rowsmain_cnt = np.unique(X_trainvalid[:,8,:], axis=0,return_counts=True)
np.unique(unique_rowsmain_cnt,return_counts=True)

(array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  45,  49,  50,  51,  52,
         54,  55,  57,  58,  59,  60,  61,  62,  63,  64,  66,  67,  68,
         69,  70,  72,  75,  83,  96, 461]),
 array([59600,  3370,   935,   287,   109,   119,   112,   106,    72,
           82,    64,    48,    30,    21,    10,     6,     6,     1,
            3,     1,     2,     1,     1,     2,     1,     4,     1,
            2,     2,     3,     3,     3,     1,     1,     1,     4,
            1,     1,     2,     1,     1,     1,     1,     1,     1,
            1]))

In [52]:
LRmodel.coef_

array([[-8.69792269e-07,  0.00000000e+00,  0.00000000e+00, ...,
        -7.28510167e-07,  2.65038999e-06,  5.56744219e-06]])

In [53]:
X_LRtrain[0:10,0:10]

array([[39.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [39.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [39.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0., -1.],
       [39.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.],
       [39.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.],
       [39.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.],
       [39.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.],
       [39.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.],
       [39.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [39.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

Other bits of proving why the signature is bad for this type of data:

*I think I'll start with showing it can be the same for different time series*

In [56]:
X_LRtrain, X_LRtest = Sig_func(X_trainvalid, X_test, K=2)

(80000, 9, 40)
(80000, 110)


In [59]:
X_LRtrain1, X_LRtest1 = Sig_func(X_trainvalid, X_test, K=1)

In [60]:
print(X_trainvalid.shape)
print(X_LRtrain1.shape)
print(9*40)

(80000, 9, 40)
(80000, 10)
360


In [69]:
unique_rows1, unique_rows1_cnt = np.unique(X_LRtrain1, axis=0,return_counts=True)

In [80]:
unique_rows2, unique_rows2_cnt = np.unique(X_LRtrain, axis=0,return_counts=True)

In [89]:
X_LRtrain3, X_LRtest3 = Sig_func(X_trainvalid, X_test, K=3)

In [90]:
print(X_LRtrain3.shape) ## much bigger

(80000, 1110)


In [91]:
unique_rows3, unique_rows3_cnt = np.unique(X_LRtrain3, axis=0,return_counts=True)

In [92]:
unique_rows3.shape ## almost all unique

(79998, 1110)

In [94]:
np.unique(unique_rows3_cnt,return_counts=True)

(array([1, 3]), array([79997,     1]))

In [81]:
unique_rows2.shape

(54788, 110)

In [95]:
np.unique(unique_rows2_cnt,return_counts=True)

(array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  45,  46,  47,  49,  50,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  65,  66,  68,  70,  72,  74,
         76,  78,  85, 106]),
 array([46726,  4355,  1442,   665,   419,   274,   172,   133,   105,
           70,    67,    39,    26,    31,    22,    24,    18,    14,
            7,    12,    10,     6,     8,    10,     6,     6,     2,
            5,     5,     7,     6,     7,     6,     4,     4,     4,
            2,     3,     5,     4,     3,     5,     6,     4,     4,
            1,     3,     2,     1,     3,     2,     1,     1,     1,
            1,     3,     1,     1,     2,     1,     2,     1,     2,
            1,     1,     1,     1,     1,     1]))

In [70]:
unique_rows_org, unique_rows_org_cnt = np.unique(X_trainvalid, axis=0,return_counts=True)

In [77]:
num_samples,num_features,num_timepoints = np.shape(X_trainvalid)
unique_rows_org, unique_rows_org_cnt = np.unique(X_trainvalid.reshape(num_samples,num_features*num_timepoints), axis=0,return_counts=True)

In [78]:
unique_rows_org.shape ## all unique 80000

(80000, 360)

In [96]:
np.unique(unique_rows1_cnt,return_counts=True)

(array([    1,     2,     3,     4,     6,     8,     9,    10,    11,
           12,    13,    14,    15,    17,    20,    21,    31,    32,
           35,    36,    37,    49,    72,    75,    82,    87,    88,
           97,    99,   102,   108,   113,   142,   211,   217,   220,
          240,   243,   253,   265,   269,   270,   309,   318,   336,
          340,   641,   863,  1749,  1816,  1850,  1890,  2141,  2563,
         5633,  5796,  7195, 42541]),
 array([4, 2, 2, 2, 1, 2, 5, 2, 2, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))

In [79]:
unique_rows_org_cnt

array([1, 1, 1, ..., 1, 1, 1])

In [58]:
print(X_trainvalid.shape)
print(X_LRtrain.shape)
print(9*40)

(80000, 9, 40)
(80000, 110)
360


In [64]:
unique_rows1.shape # 75

(75, 10)